In [1]:
import pandas as pd
import spline
import os
import numpy as np
import matplotlib.pyplot as plt
import csv
from merge_script import modify_first_column
import math
from sklearn.decomposition import PCA
import glob
import Calculate_Feature
import os
from spline import np_move_avg

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import accuracy_score
import glob
from sklearn.model_selection import train_test_split, KFold, cross_val_score, LeaveOneOut
from sklearn.manifold import TSNE
from sklearn import svm, tree
import warnings
warnings.filterwarnings("ignore")


def use_pca(featuer_data, n_components=5):
    pca = PCA(n_components)
    axis_fea = pca.fit_transform(featuer_data)  # 每个样本降为n_components维
    fea = []
    for raw in range(axis_fea.shape[0]):
        for ele in axis_fea[raw, :]:
            fea.append(ele)
    return fea


def axis_normalization(datalist, stage=1):
    normalized_data = []
    #datalist = np.array(datalist)
    for data in datalist:
        maximum = max(data)
        minimum = min(data)
        # print("maximum is:",maximum,"minimum is:",minimum)
        normalized_data.append([(item - minimum) / (maximum - minimum) * stage for item in data])
    return np.array(normalized_data)


def three_fuse(data):
    axis_num = int(data.shape[0] / 3)
    f_nd = []

    for f_in in range(axis_num):
        nd = []
        index = f_in * 3
        for i in range(data.shape[1]):
            d = data[:, i]
            fuse_data = math.sqrt(d[index] * d[index] + d[index + 1] * d[index + 1] + d[index + 2] * d[index + 2])
            nd.append(fuse_data)
        f_nd.append(nd)
    return f_nd



In [2]:
l_rf = RandomForestClassifier(n_estimators=100, random_state=42)
# l_svc = svm.SVC(C=1.0, kernel='rbf')
l_svc = svm.SVC(C=1000, kernel='rbf')
l_clftree = tree.DecisionTreeClassifier(criterion='entropy',random_state=42)

# loo = LeaveOneOut()
ID = range(1,15)
real_total_file = []
total_file = []

real_path = 'datasets/USC-HAD-clean/Subject'
jump_path_virtual3 = 'datasets/Merge_fold/ByNum_spline3/'
jump_path_virtual4 = 'datasets/Merge_fold/ByNum_spline4/'
jump_path_virtual5 = 'datasets/Merge_fold/ByNum_spline5/'

# total_file.append(jump_path_virtual)

for index in range(5):
    total_file.append(jump_path_virtual3+str(index))
    
for index in range(5):
    total_file.append(jump_path_virtual4+str(index))


for index in range(5):
    total_file.append(jump_path_virtual5+str(index))

for index in ID:
    total_file.append(real_path + str(index))
print(total_file)
print(len(total_file))

['datasets/Merge_fold/ByNum_spline3/0', 'datasets/Merge_fold/ByNum_spline3/1', 'datasets/Merge_fold/ByNum_spline3/2', 'datasets/Merge_fold/ByNum_spline3/3', 'datasets/Merge_fold/ByNum_spline3/4', 'datasets/Merge_fold/ByNum_spline4/0', 'datasets/Merge_fold/ByNum_spline4/1', 'datasets/Merge_fold/ByNum_spline4/2', 'datasets/Merge_fold/ByNum_spline4/3', 'datasets/Merge_fold/ByNum_spline4/4', 'datasets/Merge_fold/ByNum_spline5/0', 'datasets/Merge_fold/ByNum_spline5/1', 'datasets/Merge_fold/ByNum_spline5/2', 'datasets/Merge_fold/ByNum_spline5/3', 'datasets/Merge_fold/ByNum_spline5/4', 'datasets/USC-HAD-clean/Subject1', 'datasets/USC-HAD-clean/Subject2', 'datasets/USC-HAD-clean/Subject3', 'datasets/USC-HAD-clean/Subject4', 'datasets/USC-HAD-clean/Subject5', 'datasets/USC-HAD-clean/Subject6', 'datasets/USC-HAD-clean/Subject7', 'datasets/USC-HAD-clean/Subject8', 'datasets/USC-HAD-clean/Subject9', 'datasets/USC-HAD-clean/Subject10', 'datasets/USC-HAD-clean/Subject11', 'datasets/USC-HAD-clean/Sub

In [15]:
class CustomLeaveOneOut:
    def __init__(self, start=7, end=13):
        self.start = start
        self.end = end
        self.current = start

    def __iter__(self):
        return self

    def __next__(self):
        if self.current > self.end:
            raise StopIteration
        test_indices = [self.current]
        train_indices = list(range(self.start, self.end + 1))
        train_indices.remove(self.current)
        self.current += 1
        # return list(range(47)) + train_indices, test_indices
        # return train_indices, test_indices
        # return list(range(8,9))+ list(range(2,6)), test_indices
        # return list(range(3,5)) + list(range(1)) + list(range(6,7)), test_indices
        return list(range(5,15)) + train_indices, test_indices
        # return list(range(15)) + train_indices, test_indices

In [16]:
""" 1-WALK 6-RUN 7-JUMP 8-SIT """
# Real_motion_type = ['a1t', 'a6t', 'a7t', 'a8t']
# Real_motion_type = ['a1t', 'a7t', 'a8t'] # a little
Real_motion_type = ['a6t', 'a7t', 'a8t'] # a little
# encode_feauture = [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0] # a little
encode_feauture = [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]
label = [0, 1, 2, 3, 4, 5, 6, 7]

loo = CustomLeaveOneOut(start=15, end=28)
# loo = CustomLeaveOneOut(start=7, end=13)
# loo = CustomLeaveOneOut(start=14, end=20)
# loo = CustomLeaveOneOut(start=8, end=14)
# loo = CustomLeaveOneOut(start=40, end=46)
rf_predicted = []
svm_predicted = []
decisiontree_predicted = []
label_save_list = []
predict_SVM_save_list = []
predict_RF_save_list = []
predict_DT_save_list = []


# for train_index, test_index in loo.split(total_file):
#     print(train_index,test_index)
for train_index, test_index in loo:
    print(train_index,test_index)
    Data_X = []
    Label_X = []
    TestData_X = []
    TestLabel_X = []
    true_label = []
    for tmtrain in train_index:
        # print(tmtrain)
        subject = total_file[tmtrain]  # for each subject
        # print(subject)
        #subject_file = glob.glob(os.path.join(subject,'*.csv'))
        for i in range(len(Real_motion_type)):
            motion = Real_motion_type[i]
            motion_file = glob.glob(os.path.join(subject + '/' + '*' + motion + '*', '*.csv'))
            # motion_file = os.path.join(subject + '/' + 'motion', '*.csv')
            # motion_file = glob.glob(os.path.join(subject + '/' + 'motion/', '*.csv'))
            # print(subject + '/' +  motion )
            # print(motion_file)
            for motion_frame in motion_file:
                #print(motion_frame)
                # df = pd.read_csv(motion_frame, index_col=False, header=None,  usecols= [3,4,5,6,7,8,9,10,11,12,13,14])
                # df = pd.read_csv(motion_frame, index_col=False, header=None, usecols= [9,10,11,12,13,14])#1,2,3,4,5,6,7,8,9,10,11,12
                df = pd.read_csv(motion_frame, index_col=0, header=0, usecols= [0,1,2,3])#0,1,2,3,4,5,6,7,8,9,10,11,12
                # df = pd.read_csv(motion_frame, index_col=False, header=None, usecols= [9])#0,1,2,3,4,5,6,7,8,9,10,11,12
                # df = pd.read_csv(motion_frame, index_col=False, header=None,  usecols= [12,13,14])  #1,2,3,4,5,6,7,8,9,10,11,12
                df = np.array(df).T


                # timestamp = []
                # for t in range(df.shape[1]):
                #     timestamp.append(t*0.05)
                # timestamp = np.array(timestamp)

                # for k in range(df.shape[0]):
                #     df[k, :] = np_move_avg(df[k, :], 15, mode='same')

                # fuse three axis-data
                fuse_df = three_fuse(df)
                nor_df = axis_normalization(fuse_df)  #.tolist()
                # nor_df = axis_normalization(df)  #.tolist()

                axis_fea = []
                # rd = []
                # print(nor_df.shape)
                for raw in range(nor_df.shape[0]):
                # for raw in range(df.shape[0]):
                    tmd = nor_df[raw, :]

                    # resampling = spline.spline_cal(timestamp,tmd.tolist(),10)
                    #
                    # tmd = resampling.resample()
                    # print(tmd)

                    # tmd = df[raw, :]
                    # rd.append(tmd)
                    cal_fea = Calculate_Feature.Get_Feature(tmd, encode_feauture)
                    fea = cal_fea.cal_result()
                    for f in fea:
                        axis_fea.append(f)

                # print(axis_fea)

                Data_X.append(axis_fea)
                Label_X.append(label[i])

                # print(Label_X)

    Real_Data = np.array(Data_X)
    Real_Label = np.array(Label_X)
    # print(Real_Data.shape)
    # print(Real_Label.shape)

    # for i in range(Real_Data.shape[0]):
    #     Real_Data[i] = [0 if math.isnan(x) else x for x in Real_Data[i]]
    # print("real sample size before PCA is:", Real_Data.shape)
    #
    # pca = PCA(2)
    # Real_Data = pca.fit_transform(Real_Data)

    # tsne = TSNE(2)
    # Real_Data = tsne.fit_transform(Real_Data)

    print("real sample size is:", Real_Data.shape)
    l_rf.fit(Real_Data, Real_Label)
    l_svc.fit(Real_Data, Real_Label)
    l_clftree.fit(Real_Data, Real_Label)

    for tmtest in test_index:
        # print(tmtest)
        subject = total_file[tmtest]  # for each subject
        print(subject)
        #subject_file = glob.glob(os.path.join(subject,'*.csv'))
        for i in range(len(Real_motion_type)):
            motion = Real_motion_type[i]
            motion_file = glob.glob(os.path.join(subject + '/' + '*' + motion + '*', '*.csv'))
            for motion_frame in motion_file:
                #print(motion_frame)
                # df = pd.read_csv(motion_frame, index_col=False, header=None,  usecols= [3,4,5,6,7,8,9,10,11,12,13,14])  #1,2,3,4,5,6,7,8,9,10,11,12
                df = pd.read_csv(motion_frame, index_col=0, header=0, usecols= [0,1,2,3])  #1,2,3,4,5,6,7,8,9,10,11,12
                # df = pd.read_csv(motion_frame, index_col=False, header=None,  usecols= [9])  #1,2,3,4,5,6,7,8,9,10,11,12
                # df = pd.read_csv(motion_frame, index_col=False, header=None,  usecols= [12,13,14])  #1,2,3,4,5,6,7,8,9,10,11,12
                # df = pd.read_csv(motion_frame, index_col=False, header=None,  usecols= [9,10,11,12,13,14])  #1,2,3,4,5,6,7,8,9,10,11,12
                # df = pd.read_csv(motion_frame, index_col=False, header=None)  #1,2,3,4,5,6,7,8,9,10,11,12
                df = np.array(df).T


                # timestamp = []
                # for t in range(df.shape[1]):
                #     timestamp.append(t*0.05)
                # timestamp = np.array(timestamp)

                #
                # for k in range(df.shape[0]):
                #     df[k, :] = np_move_avg(df[k, :], 20, mode='same')

                #fuse three axis-data
                fuse_df = three_fuse(df)
                nor_df = axis_normalization(fuse_df)  #.tolist()
                # nor_df = axis_normalization(df)  #.tolist()

                axis_fea = []
                # rd = []
                for raw in range(nor_df.shape[0]):
                # for raw in range(df.shape[0]):
                    tmd = nor_df[raw, :]
                    # tmd = df[raw, :]


                    # resampling = spline.spline_cal(timestamp,tmd.tolist(),10)
                    #
                    # tmd = resampling.resample()


                    # rd.append(tmd)
                    cal_fea = Calculate_Feature.Get_Feature(tmd, encode_feauture)
                    fea = cal_fea.cal_result()
                    for f in fea:
                        axis_fea.append(f)



                TestData_X.append(axis_fea)
                TestLabel_X.append(label[i])


    TestData = np.array(TestData_X)

    # for i in range(TestData.shape[0]):
    #     TestData[i] = [0 if math.isnan(x) else x for x in TestData[i]]

    TestLabel = np.array(TestLabel_X)
    label_save_list.append(TestLabel)

    # print("test sample size before PCA is:", TestData.shape)

    # pca = PCA(2)
    # TestData = pca.fit_transform(TestData)

    # tsne = TSNE(2)
    # TestData = tsne.fit_transform(TestData)

    print("test sample size is:", TestData.shape)


    predict_RF_save_list.append(l_rf.predict(TestData))
    predict_SVM_save_list.append(l_svc.predict(TestData))
    predict_DT_save_list.append(l_clftree.predict(TestData))


    rf_predicted.append(accuracy_score(TestLabel, l_rf.predict(TestData)))
    svm_predicted.append(accuracy_score(TestLabel, l_svc.predict(TestData)))
    decisiontree_predicted.append(accuracy_score(TestLabel, l_clftree.predict(TestData)))

    #true_label.append(TestLabel_X[0])

    print("*****")

[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28] [15]
real sample size is: (575, 9)
datasets/USC-HAD-clean/Subject1
test sample size is: (15, 9)
*****
[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28] [16]
real sample size is: (575, 9)
datasets/USC-HAD-clean/Subject2
test sample size is: (15, 9)
*****
[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28] [17]
real sample size is: (575, 9)
datasets/USC-HAD-clean/Subject3
test sample size is: (15, 9)
*****
[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28] [18]
real sample size is: (575, 9)
datasets/USC-HAD-clean/Subject4
test sample size is: (15, 9)
*****
[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21, 22, 23, 24, 25, 26, 27, 28] [19]
real sample size is: (575, 9)
datasets/USC-HAD-clean/Subject5
test sample size is: (15, 9)
*****
[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18

In [17]:
print(rf_predicted)
print(svm_predicted)
print(decisiontree_predicted)
print(sum(rf_predicted) / len(rf_predicted))
print(sum(svm_predicted) / len(svm_predicted))
print(sum(decisiontree_predicted) / len(decisiontree_predicted))

[1.0, 0.8666666666666667, 1.0, 1.0, 1.0, 1.0, 0.9333333333333333, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9333333333333333]
[1.0, 0.8666666666666667, 1.0, 1.0, 1.0, 1.0, 0.9333333333333333, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
[1.0, 0.6666666666666666, 0.8666666666666667, 1.0, 1.0, 1.0, 0.8666666666666667, 0.9333333333333333, 0.8666666666666667, 1.0, 1.0, 1.0, 1.0, 0.7333333333333333]
0.980952380952381
0.9857142857142858
0.9238095238095239
